# Running Spectral Analysis Script

Now we will run the standalone spectral analysis script to demonstrate the same analysis workflow in a modular format.

In [19]:
# Import and run the spectral analysis script
import sys
import subprocess
import os

# Run the spectral analysis script
try:
    print("Running spectral analysis script...")
    result = subprocess.run([sys.executable, 'spectral_analysis.py'], 
                          capture_output=True, text=True, cwd=os.getcwd())
    
    # print("STDOUT:")
    print(result.stdout)
    
    #if result.stderr:
    #    print("STDERR:")
    #    print(result.stderr)
    
    # print(f"Return code: {result.returncode}")
    
except Exception as e:
    print(f"Error running script: {e}")

Running spectral analysis script...
Loading CSPEC data...
Loading response matrices...
Data loading complete!
Choose analysis mode:
1. Single time range analysis (1-2s)
2. Time evolution analysis (1-20s)

Running time evolution analysis (1-20s)...
MULTIPLE TIME RANGE SPECTRAL ANALYSIS
Analyzing time ranges from 1 to 20 seconds
Duration per range: 2 seconds
Total ranges: 20
Fitting background first...
Fitting background...

--- Time Range 1/20: 1-2s ---
✓ Fit successful!
  Amplitude: 8.545e-02
  Epeak: 291.0 keV
  Alpha: -0.40
  Beta: -2.00
  PGSTAT/DOF: 256.1/480

--- Time Range 3/20: 3-4s ---
✓ Fit successful!
  Amplitude: 3.444e-01
  Epeak: 340.1 keV
  Alpha: -0.46
  Beta: -2.39
  PGSTAT/DOF: 337.9/480

--- Time Range 5/20: 5-6s ---
✓ Fit successful!
  Amplitude: 2.427e-01
  Epeak: 364.5 keV
  Alpha: -0.69
  Beta: -2.26
  PGSTAT/DOF: 334.7/480

--- Time Range 7/20: 7-8s ---
✓ Fit successful!
  Amplitude: 2.055e-01
  Epeak: 384.4 keV
  Alpha: -0.72
  Beta: -2.96
  PGSTAT/DOF: 294.8/48

## Script Results

The spectral analysis script performs the following steps:

1. **Data Loading**: Loads CSPEC and response data from multiple detectors (n3, n6, n7, b0)
2. **Background Fitting**: Fits polynomial background models using intervals (-50, -10) and (30, 100) seconds
3. **Spectrum Extraction**: Extracts source spectrum from interval (1, 2) seconds
4. **Spectral Fitting**: Fits the Band function to the extracted spectrum
5. **Results**: Outputs spectral parameters with uncertainties and fit statistics

### Expected Output:
- Band function parameters: A, Epeak, α, β
- 90% confidence intervals for each parameter
- PGSTAT/DOF fit quality metric
- Plots of fitted spectrum with residuals

The script demonstrates the same analysis workflow as the notebook but in a modular, reusable format that can be easily adapted for other GRB events or analysis configurations.